In [1]:
import numpy as np 
import urdf2casadi.urdfparser as u2c
from urdf2casadi.geometry import plucker
from urdf_parser_py.urdf import URDF, Pose
from timeit import Timer, timeit, repeat
import casadi as cs

In [2]:
def median(lst):
    n = len(lst)
    if n < 1:
            return None
    if n % 2 == 1:
            return sorted(lst)[n//2]
    else:
            return sum(sorted(lst)[n//2-1:n//2+1])/2.0
        
def average(lst): 
    return sum(lst) / len(lst) 


In [3]:
def id_j_func():
    for j in range(njoints):
        q[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
        qdot[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
        qddot[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
    invd_j(q, qdot, qddot)
    

def id_func():
    for j in range(njoints):
        q[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
        qdot[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
        qddot[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
    invd(q, qdot, qddot)
    
    
    
def C_j_func():
    for j in range(njoints):
        q[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
        qdot[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
    C_j(q, qdot)
    

def C_func():
    for j in range(njoints):
        q[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
        qdot[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
    C(q, qdot)
    
    
    
def g_j_func():
    for j in range(njoints):
        q[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
    g_j(q)
    

def M_func():
    for j in range(njoints):
        q[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
    M(q)
    

    
def M_j_func():
    for j in range(njoints):
        q[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
        qdot[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
    M_j(q)
    

def g_func():
    for j in range(njoints):
        q[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
    g(q)
    
    

def fdaba_j_func():
    for j in range(njoints):
        q[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
        qdot[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
        tau[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
    fd_aba_j(q, qdot, tau)
    

def fdaba_func():
    for j in range(njoints):
        q[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
        qdot[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
        tau[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
    fd_aba(q, qdot, tau)
    
    
    
def fdcrba_j_func():
    for j in range(njoints):
        q[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
        qdot[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
        tau[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
    fd_crba_j(q, qdot, tau)
    

def fdcrba_func():
    for j in range(njoints):
        q[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
        qdot[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
        tau[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
    fd_crba(q, qdot, tau)

In [5]:
#storage for timing data
mediantime_g = 0
mediantime_gj = 0

mediantime_C = 0
mediantime_Cj = 0

mediantime_id = 0
mediantime_idj = 0

mediantime_M = 0
mediantime_Mj = 0

mediantime_fd_aba = 0
mediantime_fd_abaj = 0

mediantime_fd = 0
mediantime_fd_crbaj = 0

path_to_urdf = "/home/lillmaria/urdf2casadi/examples/urdf/pantilt.urdf"
#end_link = "link" + str(urdf_nr[i])
#root = 'calib_kuka_arm_base_link'
#tip = "kuka_arm_7_link"

root = 'base_link'
tip = "tilt_link"

robot = u2c.URDFparser()
robot.from_file(path_to_urdf)

jlist, names, qmax, qmin = robot.get_joint_info(root, tip)
njoints = len(jlist)


#u2c and pybullet declarations
q = [None]*njoints
qs =cs.SX.sym("qs",njoints)
qsdot =cs.SX.sym("qsdot",njoints)
qsddot =cs.SX.sym("qsddot",njoints)
taus =cs.SX.sym("qsddot",njoints)
qdot = [None]*njoints
qddot = [None]*njoints
o = [None]*njoints
tau = [None]*njoints

gravity = [0, 0, -9.81]
C = robot.get_coriolis_rnea(root, tip)
g = robot.get_gravity_rnea(root, tip, gravity)
invd = robot.get_inverse_dynamics_rnea(root, tip, gravity)
fd_aba = robot.get_forward_dynamics_aba(root, tip, gravity)
fd_crba = robot.get_forward_dynamics_crba(root, tip, gravity)
M = robot.get_inertia_matrix_crba(root, tip)

C_j = cs.jacobian(C(qs, qsdot), qs)
g_j = cs.jacobian(g(qs), qs)
M_j = cs.jacobian(M(qs), qs)
invd_j = cs.jacobian(invd(qs, qsdot, qsddot), qs)
fd_aba_j = cs.jacobian(fd_aba(qs, qsdot, taus), qs)
fd_crba_j = cs.jacobian(fd_crba(qs, qsdot, taus), qs)


g_j = cs.Function("g_j", [qs], [g_j], {"jit": True, "jit_options":{"flags":"-Ofast"}})
C_j = cs.Function("C_j", [qs, qsdot], [C_j], {"jit": True, "jit_options":{"flags":"-Ofast"}})
M_j = cs.Function("M_j", [qs], [M_j], {"jit": True, "jit_options":{"flags":"-Ofast"}})
invd_j = cs.Function("invd_j", [qs, qsdot, qsddot], [invd_j], {"jit": True, "jit_options":{"flags":"-Ofast"}})
fd_aba_j = cs.Function("fd_aba_j", [qs, qsdot, taus], [fd_aba_j], {"jit": True, "jit_options":{"flags":"-Ofast"}})
fd_crba_j = cs.Function("fd_crba_j", [qs, qsdot, taus], [fd_crba_j], {"jit": True, "jit_options":{"flags":"-Ofast"}})

/home/lillmaria/urdf2casadi/examples/urdf/pantilt.urdf


In [7]:
n_invd = invd.n_nodes()
n_C = C.n_nodes()
n_M = M.n_nodes()
n_g = g.n_nodes()
n_fd_aba = fd_aba.n_nodes()
n_fd_crba = fd_crba.n_nodes()

n_invdj = invd_j.n_nodes()
n_Cj = C_j.n_nodes()
n_Mj = M_j.n_nodes()
n_gj = g_j.n_nodes()
n_fd_abaj = fd_aba_j.n_nodes()
n_fd_crbaj = fd_crba_j.n_nodes()

print n_invdj
print n_Cj
print n_Mj
print n_gj
print n_fd_abaj
print n_fd_crbaj




print n_invd
print n_C
print n_M
print n_g
print n_fd_aba
print n_fd_crba
    

166
130
44
29
257
259
110
79
34
18
142
132


In [86]:
nitr = 1
nrepeat = 10000

timeit_idj = repeat("id_j_func()", setup = "from __main__ import id_j_func", repeat = nrepeat, number = nitr)
mediantime_idj = median(timeit_idj)

timeit_gj = repeat("g_j_func()", setup = "from __main__ import g_j_func", repeat = nrepeat, number = nitr)
mediantime_gj = median(timeit_gj)

timeit_Cj = repeat("C_j_func()", setup = "from __main__ import C_j_func", repeat = nrepeat, number = nitr)
mediantime_Cj = median(timeit_Cj)

timeit_Mj = repeat("M_j_func()", setup = "from __main__ import M_j_func", repeat = nrepeat, number = nitr)
mediantime_Mj = median(timeit_Mj)

timeit_fdabaj = repeat("fdaba_j_func()", setup = "from __main__ import fdaba_j_func", repeat = nrepeat, number = nitr)
mediantime_fd_abaj = median(timeit_fdabaj)

timeit_fdcrbaj = repeat("fdcrba_j_func()", setup = "from __main__ import fdcrba_j_func", repeat = nrepeat, number = nitr)
mediantime_fd_crbaj = median(timeit_fdcrbaj)

   
print "\nMedian time for evaluating derivate of inverse dynamics:\n", mediantime_idj
print "\nMedian time for evaluating derivate of gravity:\n", mediantime_gj
print "\nMedian time for evaluating derivate of Coriolis:\n", mediantime_Cj
print "\nMedian time for evaluating derivate of inertia matrix:\n", mediantime_Mj
print "\nMedian time for evaluating derivate of forward dynamics aba:\n", mediantime_fd_abaj
print "\nMedian time for evaluating derivate of forward dynamics crba:\n", mediantime_fd_crbaj


Median time for evaluating derivate of inverse dynamics:
3.2901763916e-05

Median time for evaluating derivate of gravity:
1.4066696167e-05

Median time for evaluating derivate of Coriolis:
2.28881835938e-05

Median time for evaluating derivate of inertia matrix:
1.50203704834e-05

Median time for evaluating derivate of forward dynamics aba:
3.09944152832e-05

Median time for evaluating derivate of forward dynamics crba:
3.19480895996e-05


In [69]:
timeit_id = repeat("id_func()", setup = "from __main__ import id_func", repeat = nrepeat, number = nitr)
mediantime_id = median(timeit_id)

timeit_g = repeat("g_func()", setup = "from __main__ import g_func", repeat = nrepeat, number = nitr)
mediantime_g = median(timeit_g)

timeit_C = repeat("C_func()", setup = "from __main__ import C_func", repeat = nrepeat, number = nitr)
mediantime_C = median(timeit_C)

timeit_M = repeat("M_func()", setup = "from __main__ import M_func", repeat = nrepeat, number = nitr)
mediantime_M = median(timeit_M)

timeit_fdaba = repeat("fdaba_func()", setup = "from __main__ import fdaba_func", repeat = nrepeat, number = nitr)
mediantime_fd_aba = median(timeit_fdaba)

timeit_fdcrba = repeat("fdcrba_func()", setup = "from __main__ import fdcrba_func", repeat = nrepeat, number = nitr)
mediantime_fd_crba = median(timeit_fdcrba)

   
print "\nMedian time for evaluating derivate of inverse dynamics:\n", mediantime_id
print "\nMedian time for evaluating derivate of gravity:\n", mediantime_g
print "\nMedian time for evaluating derivate of Coriolis:\n", mediantime_C
print "\nMedian time for evaluating derivate of inertia matrix:\n", mediantime_M
print "\nMedian time for evaluating derivate of forward dynamics aba:\n", mediantime_fd_aba
print "\nMedian time for evaluating derivate of forward dynamics crba:\n", mediantime_fd_crba


Median time for evaluating derivate of inverse dynamics:
3.2901763916e-05

Median time for evaluating derivate of gravity:
1.4066696167e-05

Median time for evaluating derivate of Coriolis:
2.21729278564e-05

Median time for evaluating derivate of inertia matrix:
1.38282775879e-05

Median time for evaluating derivate of forward dynamics aba:
3.19480895996e-05

Median time for evaluating derivate of forward dynamics crba:
3.19480895996e-05
